In [1]:
import numpy as np
import random
import math
import time
import copy
import pandas as pd
import operator
from collections import defaultdict

In [2]:
class Node(object):
    def __init__(self, data = None, next_node = None, prev_node = None):
        self.data = data
        self.next_node =  next_node
        self.prev_node = prev_node

class DoublyLinkedList(object):
    def __init__(self, head = None):
        self.head = head
    
    def traverse(self):
        current_node = self.head
        while current_node != None:
            print(current_node.data)
            current_node = current_node.next_node
    
    def get_size(self):
        count = 0
        current_node = self.head
        while current_node != None:
            count += 1
            current_node = current_node.next_node
        return count
            
    def append(self, data):
        new_node = Node(data)
        current_node = self.head
        new_node.next_node = current_node
        new_node.prev_node = None
        if current_node != None:
            current_node.prev_node = new_node
        self.head = new_node
    
    def insert_end(self, data):
        new_node = Node(data)
        new_node.next = None
        if self.head == None:
            new_node.prev_node = None
            self.head = new_node
        return
    
        first_node = self.head
        while first_node.next_node:
            first_node = first_node.next_node
        first_node.next_node = new_node
        new_node.prev_node = first_node
    
    
    def delete(self, data):
        current_node = self.head
        while current_node != None:
            if current_node.data == data and current_node == self.head:
                if not current_node.next_node:
                    current_node = None
                    self.head = None
                    return
                else:
                    q = current_node.next_node
                    current_node.next_node = None
                    q.prev_node = None
                    current_node = None
                    self.head = q
                    return
            
            elif current_node.data == data:
                if current_node.next_node != None:
                    p = current_node.prev_node
                    q = current_node.next_node
                    p.next_node = q
                    q.prev_node = p
                    current_node.next_node = None
                    current_node.prev_node = None
                    current_node = None
                    return
                else:
                    p = current_node.prev_node
                    p.next_node = None
                    current_node.prev_node = None
                    current = None
                    return
            current_node =  current_node.next_node

class Bucket(object):
    def __init__(self, value):
        self.gain = value
        self.data = DoublyLinkedList()

class Bucket_Arrays(object):
    def __init__(self, maxdegree):
        self.bucket_range = maxdegree
        self.left_buckets = np.array([Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)])
        self.right_buckets = np.array([Bucket(i) for i in range(-self.bucket_range,self.bucket_range+1)])

In [3]:
def calculate_num_cuts(df):
    connected_edges = df[df['Partition']==0]['ID connected vertices'].values
    outside_partition = list(df[df['Partition']==1].index)
    connected_edges = [item in outside_partition for sublist in connected_edges for item in sublist]
    num_cuts = np.sum(connected_edges)
    return num_cuts

def initialize_gain_buckets(df, buckets):
    max_degree = np.max(df['Number of connected vertices'])
    for i in range(len(df)):
        connected_edges = df['ID connected vertices'][i]
        partition_value = df['Partition'][i]
        gain = 0
        for j in connected_edges:
            connected_partition = df['Partition'][j]
            if(partition_value != connected_partition):
                gain += 1
            else:
                gain -= 1
        df.loc[i,'Gain'] = gain
        
        if partition_value == 0:
            buckets.left_buckets[gain+max_degree].data.append(i)

        else:
            buckets.right_buckets[gain+max_degree].data.append(i)
    
    return df,buckets

def calculate_vertex_max_gain(df,bucket_choice,max_degree):
    if(bucket_choice == 0):
        vertex_max_gain = df[df['Partition']==0]['Gain'].idxmax()
    else:
        vertex_max_gain = df[df['Partition']==1]['Gain'].idxmax()
    return vertex_max_gain

def update_df_buckets(df,buckets,vertex_max_gain):
    max_degree = np.max(df['Number of connected vertices'])
    partition_value = df.loc[vertex_max_gain,'Partition']
    df.loc[vertex_max_gain,'Partition'] = int(not(partition_value))
    if df.loc[vertex_max_gain,'Partition'] == 1:
        gain = df.loc[vertex_max_gain,'Gain']
        buckets.left_buckets[gain+max_degree].data.delete(vertex_max_gain)
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain)
        
    else:
        gain = df.loc[vertex_max_gain,'Gain']
        buckets.right_buckets[gain+max_degree].data.delete(vertex_max_gain)
        df,buckets = re_calculate_gain(df, buckets,vertex_max_gain)
    return df, buckets

def re_calculate_gain(df, buckets, vertex_max_gain):
    max_degree = np.max(df['Number of connected vertices'])
    vertices_to_update = df.loc[vertex_max_gain]['ID connected vertices']
    
    changed_partition = df.iloc[vertex_max_gain]['Partition']
    for i in vertices_to_update:
        if(df.loc[i]['Fixed'] == 0):
            new_gain = 0
            current_gain = df.loc[i]['Gain']
            partition_value = df.iloc[i]['Partition']
            if(partition_value == changed_partition):
                new_gain = current_gain+2
            else:
                new_gain = current_gain-2
            
            df.loc[i,'Gain'] = new_gain
            if(partition_value == 1): 
                buckets.right_buckets[new_gain+max_degree].data.append(i)
            else:
                buckets.left_buckets[new_gain+max_degree].data.append(i)
    df.loc[vertex_max_gain,'Fixed'] = 1
    df.loc[vertex_max_gain,'Gain'] = -999
    return df, buckets

def initialise_data(partitioning):
    # Loading the single planar graph of 500 vertices
    data = defaultdict(list)
    for line in open("Graph500.txt"):
        split_line=line.split()
        ID_vertex = int(split_line[0])
        num_connected_vertices  = int(split_line[2])
        ID_connected_vertices = [int(i)-1 for i in split_line[3:]]
        if (ID_vertex) not in data.keys():
            data[ID_vertex].append(int(num_connected_vertices))
            data[ID_vertex].append(0)
            data[ID_vertex].append(0)
            data[ID_vertex].append(ID_connected_vertices)
            data[ID_vertex].append(0)
    data_frame = pd.DataFrame(data.values(),columns = ['Number of connected vertices','Gain', 'Fixed','ID connected vertices', 'Partition'])
    
    num_vertices = len(data_frame)
    if(partitioning is None):
        partition = random.sample(range(0,num_vertices),num_vertices//2)
        data_frame.loc[partition,'Partition'] = 1
    else:
        data_frame["Partition"] = partitioning
    return data_frame

def FM_one_pass(df):
    num_cuts = calculate_num_cuts(df)
    num_vertices = len(df)
    min_cuts = num_cuts
    max_degree = np.max(df['Number of connected vertices'])
    buckets = Bucket_Arrays(max_degree)
    df,buckets = initialize_gain_buckets(df, buckets)
    save_partition = copy.deepcopy(df['Partition'].values)
    while(np.sum(df['Fixed']) < num_vertices):
        if(len(df[df['Partition']==0]) >= len(df[df['Partition']==1])):
            vertex_max_gain = calculate_vertex_max_gain(df,0,max_degree)
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain)           
        else:
            vertex_max_gain = calculate_vertex_max_gain(df,1,max_degree)
            df, buckets = update_df_buckets(df,buckets,vertex_max_gain)
            
        num_cuts = calculate_num_cuts(df)
        print(num_cuts)
        if(num_cuts < min_cuts and len(df[df['Partition']==0]) == len(df[df['Partition']==1])):
            save_partition = copy.deepcopy(df['Partition'].values)
            min_cuts = num_cuts
    return min_cuts, save_partition

In [4]:
optimal_partition = None
df = initialise_data(optimal_partition)

current_num_cuts = calculate_num_cuts(df)
start = time.time()
local_optimum, optimal_partition = FM_one_pass(df)
end = time.time()
elpased_time = end - start
print(elpased_time)

666
660
655
650
645
640
639
634
630
625
625
623
619
614
612
611
607
606
602
601
601
596
592
591
591
589
585
584
591
590
594
593
599
594
598
594
601
597
597
597
597
595
595
596
596
596
600
600
599
595
591
587
587
584
584
585
591
588
587
591
591
595
595
596
596
593
589
596
596
600
601
601
602
602
599
604
601
602
606
603
600
597
601
602
603
603
600
597
600
603
604
604
601
602
606
611
615
616
613
614
611
608
608
605
605
606
607
607
604
609
606
607
608
605
605
605
609
609
614
611
612
612
613
613
610
611
612
613
614
615
615
616
617
618
619
616
620
621
619
616
610
611
613
614
614
618
616
617
619
616
616
620
621
622
624
624
626
627
629
627
628
630
630
632
633
635
640
638
639
641
641
639
639
639
640
640
644
645
645
643
647
649
650
648
649
651
653
651
652
653
654
660
662
660
661
663
664
664
666
671
673
673
671
675
675
675
676
674
676
674
676
678
676
677
679
680
682
679
681
683
685
683
685
683
685
682
680
678
676
677
679
680
678
676
674
679
677
678
680
678
679
677
678
681
683
686
688
687
688
687


In [4]:
def FM_one_run(max_passes):
    flag = 0
    total_passes = 0
    total_time = 0
    best_local_optimum = math.inf
    optimal_partition = None
    while(True):
        df = initialise_data(optimal_partition)
        current_num_cuts = calculate_num_cuts(df)
        start = time.time()
        local_optimum, optimal_partition = FM_one_pass(df)
        total_passes += 1
        end = time.time()
        elpased_time = end - start
        total_time += elpased_time

        if(local_optimum < best_local_optimum and total_passes <= max_passes):
            best_local_optimum = local_optimum
            flag = 0
        else:
            flag = 1
        if(flag == 1):
            break
    converged_local_optimum = best_local_optimum
    return [converged_local_optimum, total_time, total_passes] 

In [5]:
# FM_baseline
max_passes = 10000
total_runs = 25
run_data_frame = pd.DataFrame(columns = ['Coverged local optima','Time(s)','Number of passes'])
for i in range(0,25): 
    observations = FM_one_run(max_passes)
    run_data_frame.loc[len(run_data_frame)] = observations
#run_data_frame.to_csv(r'FM_baseline.csv', index = False)
#np.savetxt(r'FM_baseline_median.txt', run_data_frame.median(), fmt='%f')

In [ ]:
# nets = []
# for i in range(len(df)):
#     conencted_vertices = df.loc[i]['ID connected vertices']
#     for j in conencted_vertices:
#         edge = (i,j)
#         reverse_edge = (j,i)
#         if(edge not in nets and reverse_edge not in nets):
#             nets.append(edge)

# nets

# connected_nets_df = []
# for i in range(num_vertices):
#     connected_nets = []
#     for j in nets:
#         if i in j:
#             connected_nets.append(j)
#     connected_nets_df.append(connected_nets)

# df['Nets'] = connected_nets_df

# df.head()